In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('main_suicidal_data.csv').drop(columns='Unnamed: 0')

In [ ]:
wModel = get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [ ]:
ll = len(thai2vec)
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

In [ ]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [ ]:
X = df['Tweet']
X_arr = X.to_list()
y = df['Label (Specialist)']

In [ ]:
le = LabelEncoder()
le.fit(y)

In [ ]:
X = prepare2train(X_arr)
y = le.transform(y)

In [ ]:
y = np.array(y)
X = fill0in(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# LSTM

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vect.shape[0],vect.shape[1],name='embed'))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(6, activation="sigmoid"))
model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.1)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True

In [ ]:
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      self.model.stop_training = True
      print("\nCallback Accuracy มากกว่า 95%!")
callbacks = myCallback()

In [ ]:
model.fit(X_train,y_train,epochs=30,batch_size=16,callbacks=[callbacks])

In [ ]:
model_acc = model.evaluate(X_test,y_test)

In [ ]:
y_pred = np.argmax(model.predict(X_test),axis=1)

In [ ]:
print(classification_report(y_test, y_pred))

# Try it with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf_not_tuning))

# Tuning Param

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in range(200,2000,200)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=0, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
print(rf_random.best_estimator_)

In [ ]:
rf_tuned = RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                       n_estimators=200)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf_tuned))